In [ ]:
!pip3 install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
serper_api_key = userdata.get('SERPER_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
os.environ['SERPER_API_KEY'] = serper_api_key

In [ ]:
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    MDXSearchTool,
    SerperDevTool
)
search_tool = SerperDevTool();
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool ( file_path = '/content/resume.md')
semantic_search_resume = MDXSearchTool(mdx = '/content/resume.md')

In [ ]:
from re import search
researcher = Agent (
    role = "Tech Job Researcher",
    goal = "Make sure to do amazing analysis on "
         "job posting to help job applicants",
    backstory = (
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    verbose = True,
    tools = [scrape_tool, search_tool]
)
profiler = Agent (
    role = "Personal Profiler for Engineers",
    goal = "Do increditble research on job applicants "
         "to help them stand out in the job market",
    backstory = (
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    verbose = True,
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume]
)

resume_strategist = Agent(
    role = "Resume Strategist for Engineers",
    goal = "Find all the best ways to make a "
         "resume stand out in the job market.",
    backstory = ("With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."),
    verbose = True,
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
)
interview_preparer = Agent(
    role = "Engineering Interview Preparer",
    goal = "Create interview questions and talking points "
         "based on the resume and job requirements",
    backstory = ( "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."),
    verbose = True,
    tools = [ scrape_tool, search_tool, read_resume, semantic_search_resume],
)

In [ ]:
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)



In [ ]:
job_application_crew = Crew(
    agents = [ researcher,
              profiler,
               resume_strategist,
               interview_preparer],
    tasks = [research_task,
             profile_task,
             resume_strategy_task,
             interview_preparation_task],
    verbose = True
)

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/4035440814',
    'github_url': 'https://github.com/prxshetty',
    'personal_writeup': """Pranam is an accomplished data scientist
    specializing in managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MS and a strong
    background in AI and data science. Pranam has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
result = job_application_crew.kickoff(inputs = job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/4035440814) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


> Entering new CrewAgentExecutor chain... [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers

 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/prxshetty) URLs, and personal write-up (Pranam is an accomplished data scientist
    specializing in managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MS and a strong
    background in AI and data science. Pranam has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for